In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/OOD

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/OOD New
ls: cannot access 'data/dbpedia': No such file or directory


In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d67da0bc-3cec-610d-14cb-c90559181adb)


In [3]:
!pip install transformers &> /dev/null
!pip install datasets &> /dev/null

import numpy as np
import pandas as pd
import transformers
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torch import cuda
import re

from transformers import AutoTokenizer

import tqdm
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

import matplotlib.pyplot as plt

device = 'cuda' if cuda.is_available() else 'cpu'
torch.manual_seed(0)
np.random.seed(0)

In [4]:
import models_decoder
import models_baseline
import data_utils

In [5]:
EPOCHS = 10
LEARNING_RATE = 5e-06

# Model
model_type = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_type, use_fast=False)
hidden_dim = 64
dropout = 0.
loss = torch.nn.MSELoss()
device = device

In [6]:
train, val, test, label_indices, TOP_K = data_utils.get_data(data_utils.NEWS, tokenizer)

Using custom data configuration default
Reusing dataset news_category (/root/.cache/huggingface/datasets/Fraser___news_category/default/0.0.0/737b7b6dff469cbba49a6202c9e94f9d39da1fed94e13170cf7ac4b61a75fb9c)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached sorted indices for dataset at /root/.cache/huggingface/datasets/Fraser___news_category/default/0.0.0/737b7b6dff469cbba49a6202c9e94f9d39da1fed94e13170cf7ac4b61a75fb9c/cache-ab721d21669e2c4b.arrow
Loading cached sorted indices for dataset at /root/.cache/huggingface/datasets/Fraser___news_category/default/0.0.0/737b7b6dff469cbba49a6202c9e94f9d39da1fed94e13170cf7ac4b61a75fb9c/cache-488f1c2cd74134c6.arrow
Loading cached sorted indices for dataset at /root/.cache/huggingface/datasets/Fraser___news_category/default/0.0.0/737b7b6dff469cbba49a6202c9e94f9d39da1fed94e13170cf7ac4b61a75fb9c/cache-89c9d7dd9cecdd83.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/Fraser___news_category/default/0.0.0/737b7b6dff469cbba49a6202c9e94f9d39da1fed94e13170cf7ac4b61a75fb9c/cache-3a3c84da615e460e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/Fraser___news_category/default/0.0.0/737b7b6dff469cbba49a6202c9e94f9d39da1fed94e13170cf7ac4b61a75fb

Dataset({
    features: ['category_num', 'category', 'headline', 'authors', 'link', 'short_description', 'date', 'content'],
    num_rows: 68944
})


Loading cached processed dataset at /root/.cache/huggingface/datasets/Fraser___news_category/default/0.0.0/737b7b6dff469cbba49a6202c9e94f9d39da1fed94e13170cf7ac4b61a75fb9c/cache-c7eeca506c2cc4fd.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/Fraser___news_category/default/0.0.0/737b7b6dff469cbba49a6202c9e94f9d39da1fed94e13170cf7ac4b61a75fb9c/cache-d9b208c5175dc4a6.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/Fraser___news_category/default/0.0.0/737b7b6dff469cbba49a6202c9e94f9d39da1fed94e13170cf7ac4b61a75fb9c/cache-a1b5ea80401dcbc4.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/Fraser___news_category/default/0.0.0/737b7b6dff469cbba49a6202c9e94f9d39da1fed94e13170cf7ac4b61a75fb9c/cache-a447c3f392087a1b.arrow


In [7]:
decoder = models_decoder.DECODER_OOD(model_type,
                        hidden_dim,
                        # loss,
                        dropout,
                        label_indices,
                        TOP_K,
                        device
                        ).to(device)

/content/drive/MyDrive/Colab Notebooks/OOD New/models_decoder.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.label_index_tensor = torch.tensor(label_indices).to(device)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing B

torch.Size([5, 768])


In [8]:
optimizer = torch.optim.Adam(decoder.get_optimizer_parameters(), lr = LEARNING_RATE)

In [9]:
decoder.device

'cuda'

In [11]:
for i in range(6):
  epoch_losses  = decoder.train_model(train, optimizer, val_loader = val, test_loader = test, eval_interval = 300)

  0%|          | 0/1 [00:00<?, ?epoch/s]

  0%|          | 0/2155 [00:00<?, ?batch/s]

Iteration: 300


  0%|          | 0/314 [00:00<?, ?batch/s]

VAL: 0.7755331694933126


...

  0%|          | 0/314 [00:00<?, ?batch/s]

VAL: 0.8216706004493434


  0%|          | 0/942 [00:00<?, ?batch/s]

TEST: 0.8198961075662902


KeyboardInterrupt: ignored